# Final project guidelines

**Note:** Use these guidelines if and only if you are pursuing a **final project of your own design**. For those taking the final exam instead of the project, see the (separate) [final exam notebook](https://github.com/wilkens-teaching/info3350-s22/blob/main/final_exam/exam.ipynb).

## Guidelines

These guidelines are intended for **undergraduates enrolled in INFO 3350**. If you are a graduate student enrolled in INFO 6350, you're welcome to consult the information below, but you have wider latitude to design and develop your project in line with your research goals.

### The task

Your task is to: identify an interesting problem connected to the humanities or humanistic social sciences that's addressable with the help of computational methods, formulate a hypothesis about it, devise an experiment or experiments to test your hypothesis, present the results of your investigations, and discuss your findings.

These tasks essentially replicate the process of writing an academic paper. You can think of your project as a paper in miniature.

You are free to present each of these tasks as you see fit. You should use narrative text (that is, your own writing in a markdown cell), citations of others' work, numerical results, tables of data, and static and/or interactive visualizations as appropriate. Total length is flexible and depends on the number of people involved in the work, as well as the specific balance you strike between the ambition of your question and the sophistication of your methods. But be aware that numbers never, ever speak for themselves. Quantitative results presented without substantial discussion will not earn high marks. 

Your project should reflect, at minimum, ten or more hours of work by each participant, though you will be graded on the quality of your work, not the amount of time it took you to produce it.

#### Pick an important and interesting problem!

No amount of technical sophistication will overcome a fundamentally uninteresting problem at the core of your work. You have seen many pieces of successful computational humanities research over the course of the semester. You might use these as a guide to the kinds of problems that interest scholars in a range of humanities disciplines. You may also want to spend some time in the library, reading recent books and articles in the professional literature. **Problem selection and motivation are integral parts of the project.** Do not neglect them.

### Format

You should submit your project as a Jupyter notebook, along with all data necessary to reproduce your analysis. If your dataset is too large to share easily, let us know in advance so that we can find a workaround. If you have a reason to prefer a presentation format other than a notebook, likewise let us know so that we can discuss the options.

Your report should have four basic sections (provided in cells below for ease of reference):

1. **Introduction and hypothesis.** What problem are you working on? Why is it interesting and important? What have other people said about it? What do you expect to find?
2. **Corpus, data, and methods.** What data have you used? Where did it come from? How did you collect it? What are its limitations or omissions? What major methods will you use to analyze it? Why are those methods the appropriate ones?
3. **Results.** What did you find? How did you find it? How should we read your figures?
4. **Discussion and conclusions.** What does it all mean? Do your results support your hypothesis? Why or why not? What are the limitations of your study and how might those limitations be addressed in future work?

Within each of those sections, you may use as many code and markdown cells as you like. You may, of course, address additional questions or issues not listed above.

All code used in the project should be present in the notebook (except for widely-available libraries that you import), but **be sure that we can read and understand your report in full without rerunning the code**. Be sure, too, to explain what you're doing along the way, both by describing your data and methods and by writing clean, well commented code.

### Grading

This project takes the place of the take-home final exam for the course. It is worth 20% of your overall grade. You will be graded on the quality and ambition of each aspect of the project. No single component is more important than the others.

### Practical details

* The project is due at **11:59pm EST on Thursday, May 19, 2022** via upload to CMS of a single zip file containing your fully executed Jupyter notebook and all associated data.
* You may work alone or in a group of up to three total members.
    * If you work in a group, be sure to list the names of the group members.
    * For groups, create your group on CMS and submit one notebook for the entire group. **Each group member should also submit an individual statement of responsibility** that describes in general terms who performed which parts of the project.
* You may post questions on Ed, but should do so privately (visible to course staff only).
* Interactive visualizations do not always work when embedded in shared notebooks. If you plan to use interactives, you may need to host them elsewhere and link to them.

---

## 1. Introduction and hypothesis

## 2. Data and methods

In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import numpy as np
import os
import re
from   sklearn.feature_extraction.text import TfidfVectorizer
from   sklearn.feature_selection import SelectKBest, mutual_info_classif
from   sklearn.linear_model import LogisticRegression, LinearRegression
from   sklearn.model_selection import cross_val_score
from   sklearn.preprocessing import StandardScaler

In [3]:
def song_data(date=''):
    '''
    The Billboard Hot 100 chart represents the Hot 100 songs for that week.
    
    date: a string, in the form "YYYY-MM-DD". For example, "2022-05-16" represents May 16, 2022. If no date specified, function
          will select the present chart
    returns: a pandas dataframe containing metadata for Billboard Hot 100 songs of the week of the specified date.
    columns: rank: rank of the week (1-100)
             date: a pandas datetime object. date of the chart as stated on the Billboard website, 
             which uses the Saturday to identify the week (so it is the same week as the user input, but the Saturday
             of that week),
             title: title of the song,
             artist1: main artist,
             artist2: a list of the rest of the artists. np.nan if there are none.
             peak_pos: peak position of the song,
             wks_chart: # of weeks the song has been on the chart
             b_url: url to the billboard chart
    '''
    lsongs=[]
    lartists=[]
    artist1=[]
    artist2=[]
    lpeak_pos=[]
    lwks_chart=[]
    
    URL='https://www.billboard.com/charts/hot-100/'+date

    page=requests.get(URL)
    soup=BeautifulSoup(page.content, 'lxml')
  
    ### get the first song, bc it's in a different div container
    song1 = soup.find("h3",id='title-of-a-story', class_="c-title a-no-trucate a-font-primary-bold-s u-letter-spacing-0021 u-font-size-23@tablet lrv-u-font-size-16 u-line-height-125 u-line-height-normal@mobile-max a-truncate-ellipsis u-max-width-245 u-max-width-230@tablet-only u-letter-spacing-0028@tablet")
    lsongs.append(song1.text.strip())
    
    ### get the first artist, bc it's in a different div container
    artistf=soup.find("span", class_="c-label a-no-trucate a-font-primary-s lrv-u-font-size-14@mobile-max u-line-height-normal@mobile-max u-letter-spacing-0021 lrv-u-display-block a-truncate-ellipsis-2line u-max-width-330 u-max-width-230@tablet-only u-font-size-20@tablet")
    lartists.append(artistf.text.strip())
    
    ### get the first peak position, bc it's in a different div container
    nums=soup.find_all('span', class_="c-label a-font-primary-bold-l a-font-primary-m@mobile-max u-font-weight-normal@mobile-max lrv-u-padding-tb-050@mobile-max u-font-size-32@tablet")
    nums1=[]
    for x in nums:
        nums1.append(x.text.strip())
        
    lpeak_pos.append(nums1[1])
    ### get the first weeks on chart, bc it's in a different div container
    lwks_chart.append(nums1[2])
    
    ### get last 99 songs
    songs = soup.find_all("h3", class_="c-title a-no-trucate a-font-primary-bold-s u-letter-spacing-0021 lrv-u-font-size-18@tablet lrv-u-font-size-16 u-line-height-125 u-line-height-normal@mobile-max a-truncate-ellipsis u-max-width-330 u-max-width-230@tablet-only", id="title-of-a-story")
    for song in songs:
        lsongs.append(song.text.strip())
    
    ### get last 99 artists
    artists = soup.find_all("span", class_="c-label a-no-trucate a-font-primary-s lrv-u-font-size-14@mobile-max u-line-height-normal@mobile-max u-letter-spacing-0021 lrv-u-display-block a-truncate-ellipsis-2line u-max-width-330 u-max-width-230@tablet-only")
    for artist in artists:
        lartists.append(artist.text.strip())
        
    ### get last 99 peak position
    all_num=[]
    peak_pos = soup.find_all("span", class_="c-label a-font-primary-m lrv-u-padding-tb-050@mobile-max")
    for num in peak_pos:
        all_num.append(num.text.strip())
    
    x=1
    for peak in all_num:
        if x <= len(all_num)-5:
            lpeak_pos.append(all_num[x])
            x=x+6

    ### get last 99 weeks on chart
    y=2
    for wk in all_num:
        if y <= len(all_num)-4:
            lwks_chart.append(all_num[y])
            y=y+6            
    
    ### get date as listed on the chart, aka the Saturday of the week of the user input
    date=soup.find('h2', id="section-heading")
    cdate=pd.to_datetime(date.text.strip().replace("Week of ",''))
    
    
    ### separate artists into artist1 and artist2
    for a in lartists:
        if ("X &" not in a) and ("X Featuring" not in a) and ("X /" not in a):
            a=a.replace(" X ",",")
        a=a.replace("Featuring",",")
        a=a.replace("&",",")   
        a=a.replace(" / ",",")
        List=a.split(",")
        artists = [i.strip() for i in List]
        artist1.append(artists[0])
        if len(artists)==1:
            artist2.append(np.nan)
        else:
            artist2.append(artists[1:])
    
    metadata=pd.DataFrame()
    metadata['rank']=(range(1,101)) ### get rank position
    metadata['date']=cdate
    metadata['title']=lsongs
    metadata['artist1']=artist1
    metadata['artist2']=artist2
    metadata['peak_pos']=lpeak_pos
    metadata['wks_chart']=lwks_chart
    metadata['b_url']=URL
    
    metadata=append_lyrics(metadata)
    metadata.reset_index(inplace=True, drop=True)

    return metadata

In [4]:
def append_lyrics(metadata):
    '''
    a helper function for song_data(). gets the song lyrics for a given song. appends the
    song lyrics for a song from Genius.com to a "lyrics" column.
    
    If the function cannot find the song on the Genius lyrics website, it will drop the entire observation from the dataset.
    
    metadata: a pandas dataframe, created from song_data(). at the least contains
              the title column and the artist1 column.
    returns: a pandas dataframe of the original dataframe with a lyrics column and URL to the
             Genius website the lyrics were taken from.
    '''
    all_lyrics=[]
    all_URL=[]
    title=metadata.title.values
    artist1=metadata.artist1.values
    for x in range(len(title)):
        t=title[x]
        a=artist1[x]
  
        t=re.sub(r'[^\w\s]', '', t)
        a=re.sub(r'[!$/]', '-', a)
        a=re.sub(r'["\\#%&;\()*\[\]+,.:;<=>?@^_`{|}~]', '', a) #[\\]
        URL= "https://www.genius.com/"+a.replace(' ','-')+'-'+t.replace(' ','-')+'-lyrics'
        URL=URL.replace('--','-')
        
        page=requests.get(URL)
        soup=BeautifulSoup(page.content, 'lxml')
        if 'Oops! Page not found' not in soup.text.strip():
            lyrics=soup.find_all('div', class_='Lyrics__Container-sc-1ynbvzw-6 jYfhrf')
            Lyrics = [re.sub(r"\[.*?\]",'',i.text.strip()) for i in lyrics]
            LYRICS=" ".join(Lyrics)
            all_lyrics.append(LYRICS)
            all_URL.append(URL)
        
        else: 
            #print(URL)
            metadata.drop([x], inplace=True)
    
    metadata['lyrics']=all_lyrics
    metadata['g_url']=all_URL
    return metadata

In [6]:
dates=pd.date_range(start='2000-07-15',end='2000-12-31',freq='W-SAT')
print(len(dates))
print(dates)

25
DatetimeIndex(['2000-07-15', '2000-07-22', '2000-07-29', '2000-08-05',
               '2000-08-12', '2000-08-19', '2000-08-26', '2000-09-02',
               '2000-09-09', '2000-09-16', '2000-09-23', '2000-09-30',
               '2000-10-07', '2000-10-14', '2000-10-21', '2000-10-28',
               '2000-11-04', '2000-11-11', '2000-11-18', '2000-11-25',
               '2000-12-02', '2000-12-09', '2000-12-16', '2000-12-23',
               '2000-12-30'],
              dtype='datetime64[ns]', freq='W-SAT')


In [15]:
Dates=[date.strftime('%Y-%m-%d') for date in dates]

In [16]:
songs=[]
for date in Dates:
    songs.append(song_data(date))
    print(date)

2000-07-15
2000-07-22
2000-07-29
2000-08-05
2000-08-12
2000-08-19
2000-08-26
2000-09-02
2000-09-09
2000-09-16
2000-09-23
2000-09-30
2000-10-07
2000-10-14
2000-10-21
2000-10-28
2000-11-04
2000-11-11
2000-11-18
2000-11-25
2000-12-02
2000-12-09
2000-12-16
2000-12-23
2000-12-30


In [25]:
df=pd.concat(songs)
df.reset_index(inplace=True)

In [32]:
df=df.iloc[:,1:]

In [33]:
df2=pd.read_csv('df2000_0101_to_2000_0708.csv')

In [36]:
df2=df2.iloc[:,2:]

In [38]:
songs2000=pd.concat([df,df2])
songs2000.reset_index(inplace=True, drop=True)

In [40]:
songs2000.to_csv('songs2000.csv')

In [48]:
songs2000

,rank,date,title,artist1,artist2,peak_pos,wks_chart,b_url,lyrics,g_url
0,1,2000-07-15 00:00:00,Everything You Want,Vertical Horizon,NaN,1,26,https://www.billboard.com/charts/hot-100/2000-...,Somewhere there's speakingIt's already coming ...,https://www.genius.com/Vertical-Horizon-Everyt...
1,2,2000-07-15 00:00:00,Try Again,Aaliyah,NaN,1,18,https://www.billboard.com/charts/hot-100/2000-...,It's been a long time (Long time)We shouldn't ...,https://www.genius.com/Aaliyah-Try-Again-lyrics
2,3,2000-07-15 00:00:00,Be With You,Enrique Iglesias,NaN,1,16,https://www.billboard.com/charts/hot-100/2000-...,Monday night and I feel so lowI count the hour...,https://www.genius.com/Enrique-Iglesias-Be-Wit...
3,4,2000-07-15 00:00:00,I Wanna Know,Joe,NaN,4,29,https://www.billboard.com/charts/hot-100/2000-...,"YeahOh-oh, yeahAlright, oh, oh, ohIt's amazing...",https://www.genius.com/Joe-I-Wanna-Know-lyrics
4,6,2000-07-15 00:00:00,Bent,matchbox twenty,NaN,6,12,https://www.billboard.com/charts/hot-100/2000-...,And if I fall along the wayPick me up and dust...,https://www.genius.com/matchbox-twenty-Bent-ly...
...,...,...,...,...,...,...,...,...,...,...
4821,95,2000-07-08,What You Want,DMX,['Sisqo'],95,2,https://www.billboard.com/charts/hot-100/2000-...,"Y'all niggas is dead, dead!What the fuck is wr...",https://www.genius.com/DMX-What-You-Want-lyrics
4822,96,2000-07-08,You Owe Me,NAS,['Ginuwine'],59,16,https://www.billboard.com/charts/hot-100/2000-...,"Uh, it's real, it's real, it's realUh, uh, owe...",https://www.genius.com/NAS-You-Owe-Me-lyrics
4823,97,2000-07-08,Dancing Queen,A*Teens,NaN,97,1,https://www.billboard.com/charts/hot-100/2000-...,"Ooh, you can danceYou can jiveHaving the time ...",https://www.genius.com/ATeens-Dancing-Queen-ly...
4824,98,2000-07-08,Don't Call Me Baby,Madison Avenue,NaN,98,1,https://www.billboard.com/charts/hot-100/2000-...,"You and me, we have an opportunityAnd we could...",https://www.genius.com/Madison-Avenue-Dont-Cal...


## 3. Results

## 4. Discussion and conclusions

resources consulted
https://stackoverflow.com/questions/640001/how-can-i-remove-text-within-parentheses-with-a-regex
https://www.pythontutorial.net/python-regex/python-regex-sub/
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.date_range.html